# Concurrency

A **concurrent** program is one consisting of multiple threads executing "at the same" and independently.

A **thread** is an independent line of execution within a process.

The OS the first concurrent program. It needs to be a multithreaded program and so the support must be built into the OS itself.

A thread has its own set of registers (its context!) on the CPU.

A thread has its own stack.

A thread has its own instruction pointer (or program counter).

Within a process, the rest of the address space is shared amongst all its threads:
- heap
- code
- static global block

<br>
<img src="images/01-AS.png" width="500">
<br>

Having multiple stacks is not a problem because we can space them out far enough to guarantee that they can't grow into each (for example by having the free space between them be larger than the systems RAM).

## Why write concurrent programs?

If have CPU intensive programs where we can break up the work into chunks and execute it independently, we can split the workload into many threads and run the whole thing faster.

If we IO intensive programs, we can split the IO parts into threads while computations continue in the background.

Web-servers are implemented as multi-threaded programs. When a request (google query) comes into a web server, a new thread is created to handle that request while the server continues listening for more requests.

## Challenges for Concurrent Programs

Concurrent programs can be non-deterministic because the order that threads are executed is determined by the CPU scheduler.

This is especially problematic when multiple threads are accessing shared data.

If a thread is interrupted in the middle of accessing shared data, that data can get corrupted.

If our threads can get interrupted in the middle of some critical instruction  or set of instructions, then the data that they are modifying may no longer be what we expect.

## Terminology

Any set of instructions which should not be interrupted if we want our program to be correct is refered to as a **critical section**.

We want our critical sections to be **atomic**.

A set of instructions are **atomic** if they can not be interrupted.

If multiple threads end up in the same critical section at the same time, we have a **race condition**.

To avoid race conditions, to ensure that critical sections are atomic, we use **locks**.

We will explore how to use and build locks.

## How to use a lock

A **lock** is a variable (technically a struct) that only one thread can hold at a time.

If we put a lock around a critical section, we ensure that only one thread can access this critical section at a time.

We make that section mutually exclusive.

To use a lock, a program instantiates it, then attempts to hold it before entering some critical section, and finally upon leaving the critical section releases it.

```c
lock_t mutex; // create the loct, mutex for mutal exclusion
// ...
lock(&mutex); // attempt to grab the lock
sharedVariable++; // critical section
unlock(&mutex); // release the lock
```

If thread A holds the lock when thread B attempts to grab it, then Thread B must wait until A releases the lock before it can get it and continue execution.

A program can create multiple locks. If we have multiple critical sections, we want to have a lock for each critical section so that different threads can execute different critical sections simultaneously.



# Building a Lock

## Goals

**Correctness**: Our lock should correctly ensure mutualy exclusion for critical sections.

**Fairness**: If we have multiple threads all vying for the same lock, eventually each thread should be able to get the lock. We don't want to starve our threads.

**Performance**: Minimize overhead. Within a single the overhead should be minimal. Over the whole system, we want to minimize the overall time spent waiting for locks.

## A first Lock

To implement a lock, we need to implement:
1) lock initialization
2) locking it
3) unlocking it

Our simplest implementation can just be a flag.

If the flag has a value of 1, then it is held by some thread.

Otherwise, if it has a value of 0, then the lock is available.

A basic Spin Lock

```c
typedef struct lock_t {
  // data for the lock
  int flag;
} lock_t;

void init(lock_t &mutex){
  // init code
  mutex->flag = 0; // 0 -> available, 1 -> held
}

void lock(lock_t &mutex) {
  // aquire the lock
  // continue to check if the is available
  // and when it is, grab it
  while(mutex-> == 1){
      ; // spin, waiting for the lock to become free
  }
  mutex->flag = 1; // lock is free, grab it
}

void unlock(lock_t &mutex) {
  // release the lock
  mutex->flag = 0;
}
```

Our basic spin lock is NOT correct. Since the OS can interrupt `lock()` between TESTING the flag and SETTING the flag to be unavailable, multiple threads can end up with the lock.

Suppose Thread A tests the lock, sees that it is available and is interrupts after the while loop but before grabbing the flag. Then Thread B comes along, see that the lock is stil available, grabs it, enters the critical section and then is interrupted.

Thread A starts running again, grabs the lock, and enters the critical section. 

Now both threads A and B are within the critical section at the same time.

Unfortunately, we have no single C instructions that allows us to both test and update some variable atomically.

We need additional support, hardware support to enable correct locks.

### Hardware Support

There is a set of concurrency primitives which can not be interrupted which we can use to build our locks.

#### Test and Set

We need to ability to set and update our flag atomically.

Expressed as c code:

``` c
// update and return old flag atomically
int TestAndSet(int *flag, int newValue) {
  int old = *flag;
  *flag = newValue;
  return old;
```

## A correct Spin Lock

```c
typedef struct lock_t {
  // data for the lock
  int flag;
} lock_t;

void init(lock_t &mutex){
  // init code
  mutex->flag = 0; // 0 -> available, 1 -> held
}

void lock(lock_t &mutex) {
  // aquire the lock
  // continue to check if the is available
  // and when it is, grab it
  while(TestAndSet(&mutex->flag, 1)){
      ; // spin, waiting for the lock to become free
  }
}

void unlock(lock_t &mutex) {
  // release the lock
  mutex->flag = 0;
}
```

If the flag is available (flag == 0), it will be set to 1 and 0 will be returned, breaking us out to the loop, allowing us to enter the critical section.

If the flag was unavailable (flag == 1), it will be set to 1 (no change), and 1 will be returned, forcing the loop to execute again.

We have a correct solution, all operations of the shared flag variable are atomic.

## Performance Analysis

If the lock isn't available, the waiting will spin on the cpu, wasting CPU cycles. 

One solution is the **yield()** the CPU. If the lock isn't available, give up the CPU so that other threads can run.

If threads waiting on a lock, give up the CPU, then it is more likely that the thread holding the lock will get more CPU time sooner in order to release the lock sooner.

## Fairness Analysis

It is possible for a a thread to yield the CPU again and again and again, each time some other thread getting the lock.

A thread waiting on the lock can absolutely starve if it is never scheduled when the lock is free.

A Yield Lock

```c
typedef struct lock_t {
  // data for the lock
  int flag;
} lock_t;

void init(lock_t &mutex){
  // init code
  mutex->flag = 0; // 0 -> available, 1 -> held
}

void lock(lock_t &mutex) {
  // aquire the lock
  // continue to check if the is available
  // and when it is, grab it
  while(TestAndSet(&mutex->flag, 1)){
      yield(); // give up the CPU, waiting for the lock to become free
  }
}

void unlock(lock_t &mutex) {
  // release the lock
  mutex->flag = 0;
}
```